In [1]:
# import statements
import numpy as np
import dicom
import cv2 
import matplotlib.pyplot as plt
from collections import defaultdict
import scipy.misc as misc
import sys
import os
sys.path.append("..")
from AlgoEngine.utils import getMeanTargetDose, getImageBlock, getContours, convertROIToCTSpace
from General.testing_utils import getContourInputs

## Inputs to Function + Function

We combine both input and function for testing reasons- we test  this function by determining what the max dose mean is.

In [2]:
base = '/home/radiation/RadiationTherapyDecisionSupport/data/'
StudyID = 'UCLA_PR_6'
structureset = dicom.read_file(base + StudyID + '/structureset.dcm')

In [3]:
dose_data = dicom.read_file(base + StudyID + "/dose.dcm")
dose_grid = np.array(dose_data.pixel_array)

x_spacing = np.array(dose_data.PixelSpacing[0]).astype(np.uint8)
y_spacing = np.array(dose_data.PixelSpacing[1]).astype(np.uint8)

In [4]:
dose_z = float(dose_data.ImagePositionPatient[2])
ctFilenames = [fl for fl in os.listdir(base + StudyID) if 'CT.' in fl]
ct_sample = dicom.read_file(base + StudyID + '/' + ctFilenames[0])
block_shape = (ct_sample.Rows, ct_sample.Columns, len(ctFilenames))

ct_coords = None

for i, fl in enumerate(ctFilenames):
    ct_struct = dicom.read_file(base + StudyID + '/' + fl)
    if(float(ct_struct.ImagePositionPatient[2]) == dose_z):
        ct_coords = ct_struct.ImagePositionPatient
        ct_spacing = ct_struct.PixelSpacing
                            

In [5]:
x0 = float(dose_data.ImagePositionPatient[0])- float(ct_coords[0])
x0 = int(round(x0 / float(ct_spacing[0])))
y0 = float(dose_data.ImagePositionPatient[1])- float(ct_coords[1])
y0 = int(round(y0 / float(ct_spacing[1])))

DoseGridScaling = dose_data.DoseGridScaling
_, sopUID = getImageBlock(StudyID, base)

In [6]:
dose_max = -1
dose_max_name = ""
for n in range(0, len(structureset.StructureSetROISequence)):
    ROI_NAME = structureset.StructureSetROISequence[n].ROIName
    block_shape, contour_data, image_orientation, image_position, pixel_spacing = getContourInputs(base, StudyID, 
                                                                                                   ROI_NAME, excluding=[])
    
    _, ptv_roi_block = getContours(block_shape, contour_data, image_orientation, image_position, pixel_spacing)
    ct_ptv_roi_block = convertROIToCTSpace(ptv_roi_block, image_position, sopUID)
    
    dose_mean = getMeanTargetDose(ct_ptv_roi_block, block_shape, dose_grid, DoseGridScaling, x0, y0, x_spacing, y_spacing, sopUID)
    
    print(ROI_NAME + ": " + str(dose_mean))
    
    if dose_mean > dose_max and ROI_NAME != 'BODY':
        dose_max_name = ROI_NAME
        dose_max = dose_mean

Rectum: 0.0
BODY: 0.0209558
Lt Femrl Head: 0.0
PTV1_40Gy: 0.0
PTV2_35Gy: 0.000822446
PTV3_30Gy: 0.000761941
Bladder: 0.0122627
Anus: 0.0
Penile Bulb: 0.0
Urethra: 0.000646637
Rt Femrl Head: 0.000269305
Seed1: 0.0
Seed2: 0.00239527
Seed3: 0.0
Z_PTV2opti: 0.00107983
ring: 0.00150879
Combined: 0.000324989
z_PTV3opti: 0.000736779


## Testing Function

We display the max non-body dose mean (and expect that this will occur in a PTV region)

In [7]:
dose_max_name

'Bladder'